# Data Preparation

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn-white')

In [2]:
# Creo il dataframe leggendo il file csv
df = pd.read_csv('customer_supermarket.csv', sep='\t')
df.head(3)

,Unnamed: 0,BasketID,BasketDate,Sale,CustomerID,CustomerCountry,ProdID,ProdDescr,Qta
0,0,536365,01/12/10 08:26,"2,55",17850.0,United Kingdom,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6
1,1,536365,01/12/10 08:26,"3,39",17850.0,United Kingdom,71053,WHITE METAL LANTERN,6
2,2,536365,01/12/10 08:26,"2,75",17850.0,United Kingdom,84406B,CREAM CUPID HEARTS COAT HANGER,8


In [3]:
# Elimino la colonna chiamata'Unnamed: 0' perché è superflua (alla lettura del dataset, pandas ha già assegnato un indice
# ad ogni riga)
df.drop(columns=['Unnamed: 0'], inplace=True)
df.head(3)

,BasketID,BasketDate,Sale,CustomerID,CustomerCountry,ProdID,ProdDescr,Qta
0,536365,01/12/10 08:26,"2,55",17850.0,United Kingdom,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6
1,536365,01/12/10 08:26,"3,39",17850.0,United Kingdom,71053,WHITE METAL LANTERN,6
2,536365,01/12/10 08:26,"2,75",17850.0,United Kingdom,84406B,CREAM CUPID HEARTS COAT HANGER,8


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 471910 entries, 0 to 471909
Data columns (total 8 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   BasketID         471910 non-null  object 
 1   BasketDate       471910 non-null  object 
 2   Sale             471910 non-null  object 
 3   CustomerID       406830 non-null  float64
 4   CustomerCountry  471910 non-null  object 
 5   ProdID           471910 non-null  object 
 6   ProdDescr        471157 non-null  object 
 7   Qta              471910 non-null  int64  
dtypes: float64(1), int64(1), object(6)
memory usage: 28.8+ MB


Da df.info() si nota che le colonne CustomerID e ProdDescr hanno alcuni valori NaN.

### Gestione delle righe duplicate

Nel dataframe df sono presenti delle righe duplicate. Dato che il dataset sembra essere un storico degli acquisti fatti su un sito web, posso supporre che i duplicati siano dovuti ad degli errori e quindi possano essere eliminati. Escludo la possibilità che l'utente possa aggiungere al carrello lo stesso prodotto più volte (come acquisti separati). Ad esempio, su Amazon se si prova ad aggiungere al carrello lo stesso oggetto più volte, viene incrementata la quantità dell'oggetto senza aggiungere un nuovo acquisto.

In [5]:
print("Righe duplicate (esclusa la prima occorrenza) all'interno di df: ", df.duplicated().value_counts()[1])

Righe duplicate (esclusa la prima occorrenza) all'interno di df:  5232


In [6]:
df[df.duplicated()].head(3)

,BasketID,BasketDate,Sale,CustomerID,CustomerCountry,ProdID,ProdDescr,Qta
517,536409,01/12/10 11:45,"1,25",17908.0,United Kingdom,21866,UNION JACK FLAG LUGGAGE TAG,1
527,536409,01/12/10 11:45,"2,1",17908.0,United Kingdom,22866,HAND WARMER SCOTTY DOG DESIGN,1
537,536409,01/12/10 11:45,"2,95",17908.0,United Kingdom,22900,SET 2 TEA TOWELS I LOVE LONDON,1


In [7]:
df.drop_duplicates(inplace=True)

Per ogni colonna controllo che i valori siano in formato corretto, diversi da NaN, sensati, ecc.

### CustomerID

Da df.info() si nota che ci sono delle righe con CustomerID = NaN. Dato che l'analisi successiva si concentrerà sull'analisi del comportamento degli utenti, se non è possibile recuperare il CustomerID di queste righe esse andranno eliminate.

Per provare a recuperare questi CustomerID faccio così: per ogni riga $r_1$ con CustomerID = NaN, controllo nel dataframe se esiste una riga $r_2$ con $r_1$.BasketID = $r_2$.BasketID e con $r_2$.CustomerID $\neq$ NaN. Allora, posso assegnare ad $r_1$ il CustomerID di $r_2$.

In [8]:
# null contiene i BasketID delle righe che hanno CustomerID = NaN
null = df[df['CustomerID'].isnull()]['BasketID'].value_counts().index.values

# not_null contiene i BasketID delle righe che hanno CustomerID != NaN
not_null = df[pd.notnull(df['CustomerID'])]['BasketID'].value_counts().index.values

# Questa funzione mi restituisce l'intersezione di due insiemi
def intersection(lst1, lst2): 
    return list(set(lst1) & set(lst2))

intersection(list(null), list(not_null))

[]

$null \cap not\_null = \emptyset$. Non è possibile recuperare i valori di CustomerID, quindi elimino le righe che hanno CustomerID = NaN.

In [9]:
df.drop(df[df['CustomerID'].isnull()].index, inplace=True)

Controllo se tutti i valori di CustomerID sono dei float senza però la parte decimale (0.0, 1.0, 2.0, ecc.). Se tutti i valori sono senza parte decimale allora posso convertirli in int.

In [10]:
# Conto quante sono i CustomerID senza la parte decimale
df['CustomerID'].apply(float.is_integer).value_counts()

True    401605
Name: CustomerID, dtype: int64

In [11]:
# Numero di righe del dataframe df
df.shape[0]

401605

Tutte le righe hanno CustomerID senza la parte decimale, quindi posso convertirli in int.

In [12]:
df['CustomerID'] = df['CustomerID'].astype(int)

### ProdID

Controllando il dataframe, ho notato che alcune righe non rappresentano degli acquisti.

In [13]:
# isalpha restituisce True se la stringa contiene solo caratteri dell'alfabeto
df[df['ProdID'].apply(str.isalpha)]['ProdID'].value_counts()

POST    1197
M        460
D         77
DOT       16
CRUK      16
PADS       4
Name: ProdID, dtype: int64

Le righe che hanno come ProdID uno tra quelli elencati sopra non rappresentano degli acquisti, quindi le elimino.

In [14]:
# Elimino le righe che hanno come ProdID una stringa (quindi solo quelle elencate sopra)
df = df[df['ProdID'].apply(lambda s: not s.isalpha())]

Controllo se ci sono delle righe che non rappresentano degli acquisti ma hanno ProdID alfanumerici (lo faccio perchè con la funzione isalpha un ProdID = '123AAA' non sarebbe stato trovato).

In [15]:
import re

# Questa funzione mi restituisce True se la stringa contiene almeno una lettera dell'alfabeto, False altrimenti
def search_letters(the_string):
    if re.search('[a-zA-Z]', the_string) is None:
        return False
    return True

df[df['ProdID'].apply(search_letters)]['ProdID'].value_counts().index.values

array(['85123A', '85099B', '82494L', '85099C', '85099F', '84997D',
       '84970S', '47591D', '15056N', '84596B', '47590B', '47590A',
       '85049E', '84970L', '84997B', '84029E', '84029G', '47566B',
       '84997C', '85014B', '84596F', '15056BL', '84030E', '85049A',
       '85014A', '84406B', '16161P', '47559B', '85049G', '84997A',
       '84536A', '85049C', '46000S', '47504K', '48173C', '47503A',
       '16156S', '85199S', '84596G', '72351B', '51014A', '16169E',
       '47599A', '84510A', '35471D', '85034C', '75049L', '84509A',
       '15056P', '85040A', '72760B', '85184C', '84032B', '72351A',
       '16161U', '46000M', 'C2', '85132C', '47567B', '72807C', '85061W',
       '84971S', '51014C', '84078A', '79066K', '84535B', '85231B',
       '85035C', '82001S', '47599B', '84032A', '51014L', '85183B',
       '85071B', '84849D', '85049D', '85206A', '84279P', '84536B',
       '72800E', '15060B', '47593B', '85032A', '72802C', '85071A',
       '47504H', '72349B', '85032B', '84519B', '85034B'

Tra i ProdID c'è 'C2', che non rappresenta un acquisto, ma probabilmente una spesa addizionale di spedizione. Quindi elimino le righe in cui ProdID = 'C2.

In [16]:
df[df['ProdID'] == 'C2'].head(3)

,BasketID,BasketDate,Sale,CustomerID,CustomerCountry,ProdID,ProdDescr,Qta
1422,536540,01/12/10 14:05,50,14911,EIRE,C2,CARRIAGE,1
10319,537368,06/12/10 12:40,50,14911,EIRE,C2,CARRIAGE,1
10652,537378,06/12/10 13:06,50,14911,EIRE,C2,CARRIAGE,1


In [17]:
df.drop(df[df['ProdID'] == 'C2'].index, inplace=True)

Tra i ProdID c'è 'BANK CHARGES' che non rappresenta un acquisto, quindi elimino le righe in cui ProdID = 'BANK CHARGES'

In [18]:
df.drop(df[df['ProdID'] == 'BANK CHARGES'].index, inplace=True)

### ProdDescr

Controllo se ci sono ancora delle descrizioni = NaN.

In [19]:
print("Numero di righe con descrizione NaN: ", df[df['ProdDescr'].isnull()].shape[0])

Numero di righe con descrizione NaN:  0


### BasketDate

Controllo che tutti i valori della colonna BasketDate siano delle stringhe che rappresentano correttamente delle date.

In [20]:
from dateutil.parser import parse

# Questa funzione controlla se la stringa presa in input rappresenta correttamente una data
def is_date(string, fuzzy=False):
    try: 
        parse(string, fuzzy=fuzzy)
        return True

    except ValueError:
        return False

# Controllo se tutte i valori di BasketDate sono delle date valide
print(df['BasketDate'].apply(is_date).value_counts())

True    399689
Name: BasketDate, dtype: int64


Tutti i valori rappresentano delle date. Ora controllo se questi valori sono scritti nello stesso formato.

In [21]:
import datetime

def validate(date_text):
    try:
        datetime.datetime.strptime(date_text, '%d/%m/%y %H:%M')
        return True
    except ValueError:
        return False
    
print(df['BasketDate'].apply(validate).value_counts())

True    399689
Name: BasketDate, dtype: int64


Tutti i valori sono scritti seguendo lo stesso formato (giorno/mese/anno ora:minuto). Infine, converto tutte queste stringhe in formato datetime.

In [22]:
df['BasketDate'] = pd.to_datetime(df['BasketDate'], format="%d/%m/%y %H:%M")

### Sale

I valori della colonna Sale sono delle stringhe che rappresentano dei float, dove al posto del punto è stata usata la virgola.

In [23]:
# Questa funzione converte una stringa nel rispettivo float (3,5 --> 3.5)
def convert_to_float(s):
    comma_pos = s.find(',')
    if comma_pos >= 0:
        s = s[: comma_pos] + '.' + s[comma_pos+1 :]
    return float(s)

df['Sale'] = df['Sale'].apply(convert_to_float)

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 399689 entries, 0 to 471908
Data columns (total 8 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   BasketID         399689 non-null  object        
 1   BasketDate       399689 non-null  datetime64[ns]
 2   Sale             399689 non-null  float64       
 3   CustomerID       399689 non-null  int32         
 4   CustomerCountry  399689 non-null  object        
 5   ProdID           399689 non-null  object        
 6   ProdDescr        399689 non-null  object        
 7   Qta              399689 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int32(1), int64(1), object(4)
memory usage: 25.9+ MB


Tutte i valori della colonna Sale sono stati convertiti con successo a float64.

Controllo se ci sono delle righe con Sale $\leq$ 0.

In [25]:
df[df['Sale'] <= 0].head(3)

,BasketID,BasketDate,Sale,CustomerID,CustomerCountry,ProdID,ProdDescr,Qta
8122,537197,2010-12-05 14:02:00,0.0,12647,Germany,22841,ROUND CAKE TIN VINTAGE GREEN,1
28145,539263,2010-12-16 14:36:00,0.0,16560,United Kingdom,22580,ADVENT CALENDAR GINGHAM SACK,4
32806,539722,2010-12-21 13:45:00,0.0,14911,EIRE,22423,REGENCY CAKESTAND 3 TIER,10


In [26]:
print("Numero di righe con Sale <= 0: ", df[df['Sale'] <= 0].shape[0])

Numero di righe con Sale <= 0:  33


In [27]:
print("Numero di righe con Sale = 0: ", df[df['Sale'] == 0].shape[0])

Numero di righe con Sale = 0:  33


Non ci sono righe con Sale < 0. Mentre le righe con Sale = 0 potrebbero riferirsi a dei prodotti omaggio. Io sono interessato agli acquisti, quindi posso eliminare queste righe.

In [28]:
df.drop(df[df['Sale'] == 0].index, inplace=True)

### CustomerCountry

All'interno del dataframe ci sono delle righe con CustomerCountry = 'Unspecified'.

In [29]:
print("Righe con CustomerCountry = 'Unspecified': ", df[df['CustomerCountry'] == 'Unspecified'].shape[0])

Righe con CustomerCountry = 'Unspecified':  241


In [30]:
df[df['CustomerCountry'] == 'Unspecified'].head(3)

,BasketID,BasketDate,Sale,CustomerID,CustomerCountry,ProdID,ProdDescr,Qta
128523,549687,2011-04-11 13:29:00,7.95,12363,Unspecified,20685,DOORMAT RED RETROSPOT,2
128524,549687,2011-04-11 13:29:00,7.95,12363,Unspecified,22691,DOORMAT WELCOME SUNRISE,2
128525,549687,2011-04-11 13:29:00,7.95,12363,Unspecified,48116,DOORMAT MULTICOLOUR STRIPE,2


CustomerCountry = 'Unspecified' potrebbe essere un valore corretto, in quanto l'utente in fase di acquisto non ha voluto specificare lo stato in cui abita (ad esempio per motivi di privacy) e quindi lascio questi valori nel dataframe.

### Qta e BasketID

Controllo se ci sono delle righe che hanno Qta $\leq$ 0.

In [31]:
print("Numero di righe con Qta <= 0: ", df[df['Qta'] <= 0].shape[0])

Numero di righe con Qta <= 0:  8506


In [32]:
print("Numero di righe con Qta < 0: ", df[df['Qta'] < 0].shape[0])

Numero di righe con Qta < 0:  8506


Non ci sono righe con Qta = 0.

In [33]:
df[df['Qta'] < 0].head(3)

,BasketID,BasketDate,Sale,CustomerID,CustomerCountry,ProdID,ProdDescr,Qta
154,C536383,2010-12-01 09:49:00,4.65,15311,United Kingdom,35004C,SET OF 3 COLOURED FLYING DUCKS,-1
235,C536391,2010-12-01 10:24:00,1.65,17548,United Kingdom,22556,PLASTERS IN TIN CIRCUS PARADE,-12
236,C536391,2010-12-01 10:24:00,0.29,17548,United Kingdom,21984,PACK OF 12 PINK PAISLEY TISSUES,-24


Controllo se tutte le righe con Qta negativa hanno il BaskeID che inizia con la lettera C.

In [34]:
import re

# Questa funziona mi restituisce True se la stringa presa in input contiene un carattere dell'alfabeto, False altrimenti.
def search_letters(the_string):
    if re.search('[a-zA-Z]', the_string) is None:
        return False
    return True

df[df['Qta'] < 0]['BasketID'].apply(search_letters).value_counts()


True    8506
Name: BasketID, dtype: int64

Tutte le righe con Qta negativa hanno BasketID che inizia con la lettera C. Queste righe sembrano rappresentare degli ordini annullati o dei rimborsi.

Occorre ora gestire i rimborsi, ossia le righe con Qta < 0. I rimborsi vengono gestiti facendo le seguenti assunzioni:
* Per ogni ordine è possibile richiedere al massimo un rimborso.
* Un rimborso può essere applicato ad un ordine solo se la data dell'ordine è antecedente a quella del rimborso, il codice prodotto è lo stesso, il codice cliente è lo stesso e la quantità di oggetti presente nell'ordine è maggiore o uguale alla quantità in valore assoluto degli oggetti indicata nel rimborso.
* Dato un rimborso, se all'interno del dataset non è presente un ordine antecedente a quel rimborso, allora il rimborso non viene considerato.
* Dato un ordine, se ci sono più rimborsi applicabili a questo ordine, si applica quello con la quantità di oggetti in valore assoluto maggiore. Se ci sono due o più rimborsi con la stessa quantità ne uso uno tra questi.

Come prima cosa, mi salvo su un dataframe con tutti i rimborsi e uno con tutti gli acquisti.

In [35]:
rimborsi = df[df['Qta'] < 0]
# Aggiungo una colonna a rimborsi che mi indica se un determinato rimborso è stato utilizzato
lst = list()
for i in range(rimborsi.shape[0]):
    lst.insert(i, False) 
rimborsi = rimborsi.assign(Used=pd.Series(lst).values)

acquisti = df[df['Qta'] > 0]

In [36]:
# Mi creo un insieme che contiene i CustomerID dei clienti che hanno richiesto almeno un rimborso
set_customer_id_rimborsi = set(list(rimborsi['CustomerID'].values))

# Mi creo un dataframe che contiene solo le righe che si riferiscono ai clienti che hanno richiesto un rimborso
# acquisti_rimborsabili = acquisti[acquisti['CustomerID'].isin(set_customer_id_rimborsi)]

# Dizionario dove ogni coppia (chiave,valore) è (CustomerID x, rimborsi con CustomerID x)
dictonary_rimborsi = dict()

# Popolo il dizionario
for row in rimborsi.itertuples():
    customer_id = getattr(row, 'CustomerID')
    # Se ho già inserito i rimborsi relativi ad un CustomerID nel dizionario
    if customer_id in dictonary_rimborsi:
        continue
    # Inserisco la nuova coppia (chiave, valore) all'interno del dizionario
    
    dictonary_rimborsi[customer_id] = rimborsi[rimborsi['CustomerID'] == customer_id]
    
def update_row(row, dictonary_rimborsi):
    customer_id = row['CustomerID']
    qta = row['Qta']
    if customer_id not in dictonary_rimborsi:
        return qta
    basket_date = row['BasketDate']
    prod_id = row['ProdID']
    # Questo dataframe contiene tutti i rimborsi aventi CustomerID = customer_id
    rimborsi_customer = dictonary_rimborsi[customer_id]
    # Mi interessano solo i rimborsi che rispettano le assunzioni fatte sopra
    # Inoltre il rimborso non deve essere già stato usato in precedenza
    rimborsi_applicabili = rimborsi_customer[(rimborsi_customer['ProdID'] == prod_id) & (rimborsi_customer['BasketDate'] >= basket_date) & (abs(rimborsi_customer['Qta']) <= qta) & (rimborsi_customer['Used'] == False)]
    # Se non ci sono rimborsi applicabili per questo specifico ordine
    if rimborsi_applicabili.shape[0] == 0:
        return qta
    qta_min = rimborsi_applicabili['Qta'].min()
    # Prendo la riga che corrisponde al rimborso che ho appena effettuato
    rimborso = rimborsi_applicabili[rimborsi_applicabili['Qta'] == qta_min].iloc[0]
    rimborso['Used'] = True
    return qta + qta_min

qta_updated = acquisti.apply(lambda row: update_row(row, dictonary_rimborsi), axis=1)

Ora qta_updated contiene i valori aggiornati delle quantità di oggetti per ogni acquisto. Quindi occorre rimpiazzare i valori della colonna Qta (che non sono aggiornati) con quelli di qta_updated. Infine, è necessario eliminare dal dataframe 'acquisti' le righe che hanno quantità = 0, perché è come se l'utente avesse annullato l'intero ordine.

In [37]:
# Elimino dal dataframe acquisti la colonna Qta, che non rappresenta più le vere quantità acquistate dai clienti
acquisti = acquisti.drop(['Qta'], axis=1)
# Aggiungo al dataframe acquisti la colonna qta_updated, chiamandola Qta, che rappresenta le vere quantità acquistate dai clienti
acquisti = acquisti.assign(Qta=qta_updated)
# Elimino eventuali acquisti che dopo i rimborsi hanno Qta = 0, ossia è come se il cliente avesse annullato l'ordine
acquisti.drop(acquisti[acquisti['Qta'] == 0].index, inplace=True)

Ora tutti i valori della colonna BasketID del dataframe 'acquisti' sono delle stringhe che rappresentano dei numeri interi, quindi le trasformo in int.

In [38]:
acquisti['BasketID'] = acquisti['BasketID'].astype(int)

### Aggiungiamo la colonna "Spending"

Possiamo calcolare la spesa relativa ad ogni acquisto (ossia ogni riga del dataframe acquisti). Quindi, aggiungiamo una nuova colonna che rappresenta questa spesa.

In [39]:
acquisti['Spending'] = acquisti['Sale'] * acquisti['Qta']

In [40]:
acquisti.head(3)

,BasketID,BasketDate,Sale,CustomerID,CustomerCountry,ProdID,ProdDescr,Qta,Spending
0,536365,2010-12-01 08:26:00,2.55,17850,United Kingdom,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,15.30
1,536365,2010-12-01 08:26:00,3.39,17850,United Kingdom,71053,WHITE METAL LANTERN,6,20.34
2,536365,2010-12-01 08:26:00,2.75,17850,United Kingdom,84406B,CREAM CUPID HEARTS COAT HANGER,8,22.00


Il dataframe che utilizzeremo per l'analisi sarà acquisti. Esporto il dataframe acquisti in formato csv.

In [41]:
acquisti.to_csv('acquisti.csv')